<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 12th exercise: <font color="#C70039">First Reinforcement Learning Game (*Frozen Lake*) using OpenAI Gym</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: Finn Heydemann
* Date:   13.01.2024

<img src="https://launchyourintelligentapphome.files.wordpress.com/2019/05/frozenlake_legended.png?w=531" style="float: center;" width="600">

---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="ce33ff">DESCRIPTION</font>:

#### OpenAI Gym
In this exercise you will be using Python and OpenAI Gym to develop your reinforcement learning algorithm. The Gym library is a collection of environments that can be used freely with the reinforcement learning algorithms.

Gym has a ton of environments ranging from simple text based games to Atari games like Breakout and Space Invaders. The library is intuitive to use and simple to install. Just run **pip install gym** and you are good to go! The link to Gym's installation instructions, requirements, and documentation is included in the description. 

Further reading about OpenAI Gym is available under https://www.gymlibrary.dev/.
This notebook is based on this great post and notebook from [Rodolfo Mendes](https://morioh.com/p/18a96b9091d3).

#### Frozen Lake
This description of the game is copied directly from Gym's website. 

*Winter is coming. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water and die (Game over). At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend. The surface is described using a grid like the following:*

* SFFF
* FHFH
* FFFH
* HFFG

This grid is your environment! S is your (the agent's) starting point and it's safe. F represents the frozen surface and is also safe. H represents a hole and if your agent steps in a hole in the middle of a frozen lake, the game is over because the agent dies. Finally, G represents the goal, which is the space on the grid where the frisbee is located.

The agent can navigate *left, right, up, down* and the episode ends when the agent reaches the goal or falls in a hole. It receives a reward of **1** if it reaches the goal and **0** otherwise.

Here is the summary:
<img src="./images/FrozenLake.States.Rewards.png" style="float: center;" width="800">

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done by using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully 
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time. 
4. install gym into your env!
5. You will train an agent to play the *Frozen Lake* game using Q-learning and you will get a playback of how the agent does after being trained.
6. Again the task: Your agent has to navigate the grid by staying on the frozen surface without falling into any holes until it reaches the frisbee. If it reaches the frisbee, it wins with a reward of plus one. If it falls in a hole, it loses and receives no points for the entire episode.
7. Your tasks are highlighted in the notebook (see below)
---------------------------------

### Imports 
import all important libs including gym

In [1]:
import numpy as np
import gym
import random
import time
from   IPython.display import clear_output

In [2]:
print(gym.__version__)

0.26.2


### Creating the Environment
For creating your environment, just call *gym.make()* and pass a string of the name of the environment you want to set up. 
All the environments with their corresponding names you can use here are available on Gym's website (see above).
With this *env* object, you are able to query for information about the environment, sample states and actions, retrieve rewards and have your agent navigate the frozen lake. That is all made available to you conveniently with Gym.

In [3]:
env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="ansi")
env.reset()
# plt.imshow(env.render())
print(env.render())


SFFF
FHFH
FFFH
HFFG



In [4]:
env.step(2)
print(env.render())

  (Right)
SFFF
FHFH
FFFH
HFFG



/home/finn/Documents/python-venvs/aml/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


### Creating the Q-Table
Now, construct your Q-table, and initialize all the Q-values to zero for each state-action pair.
The number of rows in the table is equivalent to the size of the state space in the environment, and the number of columns is equivalent to the size of the action space (see above). You can get this information using *env.observation_space.n* and *env.action_space.n* as shown below in the code. Then, you can use this information to build the Q-table and initialize it with zeros.

In [5]:
env.reset()
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [6]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Initializing Q-Learning hyperparameters
Now, we're going to create and initialize all the parameters needed to implement the Q-learning algorithm.

First, with *num_episodes*, you define the total number of episodes you want the agent to play during training. Then, with *max_steps_per_episode*, you define a maximum number of steps that your agent is allowed to take within a single episode. So, if by the 100th step, the agent has not reached the frisbee or fallen through a hole, then the episode will terminate with the agent receiving zero points.

Next, you will set your *learning_rate* and your *discount_rate* as well, which was represented with the symbol (lambda (I think he means gamma)) in the course slides (keyword: discounted return G_t).

Now, the last four parameters are all related to the exploration-exploitation dilemma with respect to the epsilon-greedy policy. You are initializing your *exploration_rate* to **1** and setting the *max_exploration_rate* to **1** and a *min_exploration_rate* to **0.01**. The *max* and *min* are just bounds to how large or small your exploration rate can be. Remember, the exploration rate was represented with the symbol (epsilon) when discussed in the course slides.

Lastly, you will set the *exploration_decay_rate* to **0.01** to determine the rate at which the *exploration_rate* will decay.

**YOUR <font color="FFC300">TASK</font> in this exercise is as follows** (point 7 from the task list above):

All of the above parameters can change!
Your task is to create a *testplan* and tune all parameters by yourself and observe how they influence and change the performance of the algorithm. 
Make notes! They will help you during the exam.

In [7]:

def reinforcement_learning(env, 
                           n_episodes: int, 
                           max_steps_per_episode: int,
                           epsilon: float, 
                           epsilon_bounds: dict[str, float], 
                           epsilon_decay_rate:float, 
                           gamma: float, 
                           alpha: float):
    rewards = []
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    for episode in range(n_episodes): 
        state, _ = env.reset()
        done = False
        i = 0 
        cur_reward = 0
        for step in range(max_steps_per_episode): 
            if np.random.uniform(0, 1) < epsilon: 
                # print("exploring")
                action = env.action_space.sample()
            else: 
                # print("exploting")
                action = np.argmax(q_table[state, :])
            new_state, reward, done, *_ = env.step(action)
            q_table[state, action] = q_table[state, action] * (1 - alpha) + alpha * (reward + gamma * np.max(q_table[new_state, :]))
            cur_reward += reward
            state = new_state
            i += 1
            if done: 
                break
        rewards.append(cur_reward)
            
        epsilon = epsilon_bounds["lower"] + (epsilon_bounds["upper"] - epsilon_bounds["lower"]) * np.exp(-epsilon_decay_rate * episode)
    return q_table, np.array(rewards)


q_table, rewards = reinforcement_learning(env, 10000, 100, 1.0, {"upper": 1, "lower": .01}, .002, .99, 0.2)
q_table

array([[0.94148015, 0.93206535, 0.95099005, 0.94148015],
       [0.94148015, 0.        , 0.96059601, 0.95099005],
       [0.95099005, 0.970299  , 0.95099005, 0.96059601],
       [0.96059601, 0.        , 0.79622506, 0.7136787 ],
       [0.92382647, 0.82375843, 0.        , 0.94148015],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9801    , 0.        , 0.96059601],
       [0.        , 0.        , 0.        , 0.        ],
       [0.21606309, 0.        , 0.55483903, 0.90821052],
       [0.71729901, 0.98009968, 0.75069972, 0.        ],
       [0.97029425, 0.99      , 0.        , 0.97029894],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.63003475, 0.99      , 0.78422554],
       [0.98009998, 0.99      , 1.        , 0.9801    ],
       [0.        , 0.        , 0.        , 0.        ]])

In [8]:
# optimal route

done = False
current_idx = 0
env.reset()
while not done: 
    print(env.render())
    step = np.argmax(q_table[current_idx, :])
    current_idx, _, done, *_ = env.step(step)
    print(current_idx)
print(env.render())  


SFFF
FHFH
FFFH
HFFG

1
  (Right)
SFFF
FHFH
FFFH
HFFG

2
  (Right)
SFFF
FHFH
FFFH
HFFG

6
  (Down)
SFFF
FHFH
FFFH
HFFG

10
  (Down)
SFFF
FHFH
FFFH
HFFG

14
  (Down)
SFFF
FHFH
FFFH
HFFG

15
  (Right)
SFFF
FHFH
FFFH
HFFG



In [9]:
def rewards_every_n_episodes(rewards: np.ndarray, n: int):
    return rewards.reshape(-1, n).mean(axis=1)

rewards_every_n_episodes(rewards, 1000)

array([0.505, 0.933, 0.98 , 0.991, 0.989, 0.989, 0.994, 0.989, 0.99 ,
       0.994])

Add a bit of randomness to the environment: Turn on slippery as the default


In [10]:
env = gym.make("FrozenLake-v1", is_slippery=True, render_mode="ansi")
env.reset()

q_table, rewards = reinforcement_learning(env, 10_000, 1000, 1.0, {"upper": 1, "lower": .01}, .002, .99, 0.2)
q_table

array([[0.60200805, 0.49721   , 0.48048125, 0.47965473],
       [0.41983964, 0.25748426, 0.28986177, 0.50794081],
       [0.38611611, 0.3734549 , 0.33346012, 0.46328536],
       [0.27586325, 0.23659618, 0.28121685, 0.42768841],
       [0.6211513 , 0.44383977, 0.41258853, 0.44925727],
       [0.        , 0.        , 0.        , 0.        ],
       [0.16889485, 0.07076191, 0.37630933, 0.07650399],
       [0.        , 0.        , 0.        , 0.        ],
       [0.33525618, 0.49648703, 0.5192195 , 0.6813729 ],
       [0.46430056, 0.71032419, 0.45721522, 0.39744875],
       [0.72657169, 0.30664125, 0.27422158, 0.15825209],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.58235281, 0.36363441, 0.83539847, 0.47411973],
       [0.64424585, 0.90796894, 0.64789038, 0.69947259],
       [0.        , 0.        , 0.        , 0.        ]])

In [11]:
rewards_every_n_episodes(rewards, 1000)

array([0.123, 0.477, 0.709, 0.736, 0.734, 0.708, 0.77 , 0.729, 0.72 ,
       0.737])

In [12]:
## change some hyperparameters 

for gamma in [0.2, 0.5, 0.9, 0.99, 0.999999]:
    _, rewards = reinforcement_learning(env=env, 
                                        n_episodes=10_000, 
                                        max_steps_per_episode=100, 
                                        epsilon=1.0, 
                                        epsilon_bounds={"upper": 1, "lower": .01}, 
                                        epsilon_decay_rate=.002, 
                                        gamma=gamma, 
                                        alpha=0.2)
    rewards1000 = rewards_every_n_episodes(rewards, 1000)
    print(rewards1000)

[0.051 0.05  0.072 0.049 0.074 0.058 0.059 0.106 0.069 0.079]
[0.04  0.071 0.065 0.137 0.131 0.13  0.136 0.051 0.088 0.106]
[0.075 0.275 0.409 0.443 0.452 0.49  0.486 0.395 0.465 0.507]
[0.109 0.471 0.613 0.677 0.672 0.67  0.646 0.671 0.655 0.68 ]
[0.108 0.129 0.074 0.042 0.015 0.04  0.013 0.017 0.019 0.017]


In [13]:
def hyperparamter_finding(env, **hyperparameters): 
    n_params = np.array(np.meshgrid(*hyperparameters.values())).T.reshape(-1, len(hyperparameters))
    for param_set in n_params: 
        print(dict(zip(hyperparameters.keys(), param_set)))
        q_table, rewards = reinforcement_learning(env, *param_set)
        print(rewards_every_n_episodes(rewards, 1000))


hyperparamter_finding(env, 
                      n_episodes=(10_000, 50_000), 
                      max_steps_per_episode=100, 
                      epsilon=1, 
                      epsilon_bounds={"upper": 1, "lower": .01}, 
                      epsilon_decay_rate=(0.0005, 0.001, 0.002, 0.2), 
                      gamma=(0.2, 0.5, 0.9, 0.99), 
                      alpha=(0.05, 0.1, 0.2, 0.5))


{'n_episodes': 10000, 'max_steps_per_episode': 100, 'epsilon': 1, 'epsilon_bounds': {'upper': 1, 'lower': 0.01}, 'epsilon_decay_rate': 0.0005, 'gamma': 0.2, 'alpha': 0.05}
[0.02  0.029 0.048 0.049 0.055 0.06  0.039 0.055 0.063 0.071]
{'n_episodes': 50000, 'max_steps_per_episode': 100, 'epsilon': 1, 'epsilon_bounds': {'upper': 1, 'lower': 0.01}, 'epsilon_decay_rate': 0.0005, 'gamma': 0.2, 'alpha': 0.05}
[0.016 0.024 0.047 0.055 0.051 0.047 0.054 0.053 0.037 0.043 0.083 0.082
 0.105 0.052 0.056 0.068 0.033 0.055 0.088 0.101 0.063 0.043 0.045 0.035
 0.038 0.035 0.05  0.035 0.038 0.034 0.052 0.043 0.034 0.037 0.03  0.058
 0.077 0.117 0.089 0.101 0.109 0.123 0.033 0.063 0.088 0.102 0.052 0.049
 0.064 0.069]
{'n_episodes': 10000, 'max_steps_per_episode': 100, 'epsilon': 1, 'epsilon_bounds': {'upper': 1, 'lower': 0.01}, 'epsilon_decay_rate': 0.001, 'gamma': 0.2, 'alpha': 0.05}
[0.027 0.063 0.06  0.052 0.035 0.057 0.098 0.068 0.085 0.073]
{'n_episodes': 50000, 'max_steps_per_episode': 100, 'ep

## Results: 

When Epsilon Decay Rate is too high the agent will exploit too much and does'nt do enough exploring. He then isn't able to make it to the end, because he turns upwards or left at the beginning. If Epsilon decay rate is too low the episodes are not enough to get a prober result because the agent will not get towrds exploiting enough but is still exploring. 

The highest average rewards is received in the last 1000 episodes with the following hyperparamters: {'n_episodes': 50000, 'max_steps_per_episode': 100, 'epsilon': 1, 'epsilon_bounds': {'upper': 1, 'lower': 0.01}, 'epsilon_decay_rate': 0.002, 'gamma': 0.99, 'alpha': 0.05}. Here we have a low learning rate meaning that the q-table isn't updated very quickly. Also we have a pretty low epsilon_decay_rate meaning that the agent will lean towards exploting pretty late and a high gamma which values future rewards high. Plus many episodes. 




Create a list to hold all of the rewards you will get from each episode. 
By means of this you can observe how your game score changes over time.

In [14]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.2
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.005

rewards_all_episodes = []

In the following code section, the entire Q-learning algorithm is implemented as discussed in detail in the AML course. 
When this code is executed, this is exactly where the training will take place. 
* The first for-loop contains everything that happens within a single episode. 
* The second nested loop contains everything that happens for a single time-step.

Read all the red comments, as they contain lots of important information on the implementation.

In [15]:
# Q-learning algorithm

# loop: for a single episode
for episode in range(num_episodes):
    # initialize 'new episode' parameters
    state, info = env.reset()
    ''' The done variable just keeps track of whether or not your episode is finished.
    Initialize it to False when first starting the episode and you will see later where 
    it will get updated to notify you when the episode is over.'''
    done = False
    
    ''' Keep track of the rewards within the current episode as well.
    Hence, set rewards_current_episode = 0 since you start 
    with no rewards at the beginning of each episode.'''
    rewards_current_episode = 0

    # nested loop: for a single time-step
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        '''For each time-step within an episode set your exploration_rate_threshold 
        to a random number between 0 and 1. This will be used to determine whether 
        your agent will explore or exploit the environment in this time-step.'''
        if random.uniform(0, 1) > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()

        # Take new action
        '''After action is chosen, take that action by calling step() on your env object and 
        pass your action to it. The function step() returns a tuple containing the new state, 
        the reward for the action you took, whether or not the action ended the episode and 
        diagnostic information regarding the environment (helpful for debugging).'''
        new_state, reward, done, truncated, info = env.step(action)

        # Update Q-table for Q(s,a)
        '''Compare this implementation with the equation in the course slides.'''
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        '''Set your current state to the new_state that was returned when taking the last action 
        and then update the rewards from your current episode by adding the reward you received 
        for your previous action.'''
        # Set new state
        state = new_state
        # Add new reward 
        rewards_current_episode += reward 
        '''Then, check to see if your last action ended the episode 
        (game over by agent stepping in a hole or reaching the goal)! 
        If the action did end the episode, then jump out of this loop and start the next episode.
        Otherwise, transition to the next time-step.'''
        if done: 
            break
            
    # Exploration rate decay
    '''Once an episode is finished, you need to update your exploration_rate using exponential decay, 
    which just means that the exploration rate decays at a rate proportional to its current value. 
    You can decay the exploration_rate using the formula above, which makes use of all the exploration 
    rate parameters that were defined above in the hyperparameter section.'''
    
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list and move on to the next episode
    rewards_all_episodes.append(rewards_current_episode)

np.array(rewards_all_episodes)

array([0., 0., 0., ..., 1., 0., 1.])

### All episodes training completed
After all episodes are finished you now just calculate the average reward per thousand episodes from your list that contains the rewards for all episodes so that you can print it out and see how the rewards changed over time.

In [16]:
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.34700000000000025
2000 :  0.6520000000000005
3000 :  0.7000000000000005
4000 :  0.6370000000000005
5000 :  0.6610000000000005
6000 :  0.6840000000000005
7000 :  0.6430000000000005
8000 :  0.6900000000000005
9000 :  0.6630000000000005
10000 :  0.6660000000000005


### Interpretation

From this print, you can see that the average reward per thousand episodes did indeed progress over time. When the algorithm first started training, the first thousand episodes only averaged a reward of almost **0.18**, but by the time it got to its last thousand episodes, the reward drastically improved to almost **0.7**.

Let's take a second to understand how you can interpret these results. Your agent played **10000** episodes. At each time step within an episode, the agent received a reward of **1** if it reached the frisbee, otherwise, it received a reward of **0**. If the agent did indeed reach the frisbee, then the episode finished at that time-step.

Hence, that means for each episode, the total reward received by the agent for the entire episode is either **1** or **0**. So, for the first thousand episodes, you can interpret this score as meaning that **18%** of the time the agent received a reward of **1** and won the episode. And by the last thousand episodes from a total of **10000**, the agent was winning almost **70%** of the time.

By analyzing the grid of the game, you can see it is a lot more likely that the agent would fall in a hole or perhaps reach the max time steps than it is to reach the frisbee, so reaching the frisbee **70%** of the time is not too bad, especially since the agent had no explicit instructions to reach the frisbee. It learned that this is the correct thing to do.

* SFFF
* FHFH
* FFFH
* HFFG

At last, print out your updated Q-table to see how that has transitioned from its initial state of all zeros.

In [17]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[0.53838499 0.50578313 0.52420846 0.48233792]
 [0.32047055 0.31986608 0.43377636 0.53869119]
 [0.37556151 0.40564378 0.4153974  0.48623833]
 [0.39649431 0.16688594 0.30630989 0.46953826]
 [0.56854119 0.4534341  0.2583359  0.42517826]
 [0.         0.         0.         0.        ]
 [0.30754445 0.09814265 0.12244522 0.07138102]
 [0.         0.         0.         0.        ]
 [0.25375128 0.20912951 0.36267401 0.61580683]
 [0.4056872  0.64718252 0.25597281 0.55027423]
 [0.64662542 0.49437792 0.42056282 0.2208362 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.50564835 0.51242791 0.73628606 0.40880967]
 [0.71326707 0.87848523 0.71581137 0.74432412]
 [0.         0.         0.         0.        ]]
